# wrds_compa

> WRDS Compustat Annual

In [ ]:
#| default_exp wrds_compa

In [ ]:
#|exports
from __future__ import annotations
from typing import List, Dict, Tuple, Callable
import os

import pandas as pd
import numpy as np
import wrds 

import pandasmore as pdm
from finsets import wrds_utils, wrds_links

In [ ]:
#| eval: false
db = wrds.Connection(wrds_username=os.getenv('WRDS_USERNAME'))

Loading library list...
Done


In [ ]:
#| eval: false
db.describe_table('comp','funda')

Approximately 879274 rows in comp.funda.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),None
1,datadate,True,DATE,None
2,fyear,True,DOUBLE_PRECISION,None
3,indfmt,True,VARCHAR(12),None
4,consol,True,VARCHAR(2),None
...,...,...,...,...
943,au,True,VARCHAR(8),None
944,auop,True,VARCHAR(8),None
945,auopic,True,VARCHAR(1),None
946,ceoso,True,VARCHAR(1),None


In [ ]:
#| eval: false
db.close()

In [ ]:
#| export
def default_raw_vars():
    """Takes about 2 min to download all available data on these variables"""
    return ['datadate', 'gvkey', 'cusip' ,'cik' ,'tic' ,'fyear' ,'fyr' ,'naicsh', 'sich' ,'exchg',  
            'lt' ,'at' ,'txditc' ,'pstkl' ,'pstkrv' ,'pstk' ,'csho' ,'ajex' , 'rdip',
            'act' ,'dvc' ,'xad','seq' ,'che' ,'lct' ,'dlc' ,'ib' ,'dvp' ,'txdi' ,'dp' ,
            'txp' ,'oancf' ,'ivncf' ,'fincf' ,'dltt' ,'mib','ceq' ,'invt' ,'cogs' , 'revt',
            'sale' ,'capx' ,'xrd' ,'txdb' ,'prcc_f' ,'sstk' ,'prstkc' ,'dltis' ,'dltr' ,'emp' ,
            'dd1' ,'ppegt' ,'ppent' ,'xint' ,'txt' ,'sppe' ,'gdwl' ,'xrent' ,'re' ,'dvpsx_f' ,
            'tstk' ,'wcap' ,'rect' ,'xsga' ,'aqc' ,'oibdp' ,'dpact' ,'fic' ,'ni' ,'ivao' ,'ivst' ,
            'dv' , 'intan' ,'pi' ,'txfo' ,'pifo' ,'xpp' ,'drc' ,'drlt' ,'ap' ,'xacc' ,'itcb']             

In [ ]:
print(default_raw_vars())

['datadate', 'gvkey', 'cusip', 'cik', 'tic', 'fyear', 'fyr', 'naicsh', 'sich', 'exchg', 'lt', 'at', 'txditc', 'pstkl', 'pstkrv', 'pstk', 'csho', 'ajex', 'rdip', 'act', 'dvc', 'xad', 'seq', 'che', 'lct', 'dlc', 'ib', 'dvp', 'txdi', 'dp', 'txp', 'oancf', 'ivncf', 'fincf', 'dltt', 'mib', 'ceq', 'invt', 'cogs', 'revt', 'sale', 'capx', 'xrd', 'txdb', 'prcc_f', 'sstk', 'prstkc', 'dltis', 'dltr', 'emp', 'dd1', 'ppegt', 'ppent', 'xint', 'txt', 'sppe', 'gdwl', 'xrent', 're', 'dvpsx_f', 'tstk', 'wcap', 'rect', 'xsga', 'aqc', 'oibdp', 'dpact', 'fic', 'ni', 'ivao', 'ivst', 'dv', 'intan', 'pi', 'txfo', 'pifo', 'xpp', 'drc', 'drlt', 'ap', 'xacc', 'itcb']


In [ ]:
#| export
def download(vars: List[str]=None, # If None, downloads `default_raw_vars`
             wrds_username: str=None, #If None, looks for WRDS_USERNAME with `os.getenv`, then prompts you if needed
             ) -> pd.DataFrame:
    """Downloads `vars` from WRDS comp.funda library and adds PERMNO and PERMCO as in CCM"""

    if vars is None: vars = default_raw_vars()
    vars = ','.join(['a.gvkey', 'a.datadate'] + 
                    [f'a.{x}' for x in vars if x not in ['datadate', 'gvkey']])

    sql_string=f"""SELECT b.lpermno as permno, b.lpermco as permco, b.liid as iid, {vars}
                    FROM comp.funda a
                    INNER JOIN crsp.ccmxpf_lnkhist  b ON a.gvkey = b.gvkey
                    WHERE datadate BETWEEN b.linkdt AND COALESCE(b.linkenddt, CURRENT_DATE)
                            AND b.linktype IN ('LU','LC') AND b.linkprim IN ('P','C')
                            AND indfmt='INDL' AND datafmt='STD' AND popsrc='D' AND consol='C'"""
    
    return wrds_utils.download(sql_string, wrds_username)

In [ ]:
#| eval: false
raw = download()

Loading library list...
Done


In [ ]:
#| eval: false
raw.head(0)

,permno,permco,iid,gvkey,datadate,cusip,cik,tic,fyear,fyr,...,intan,pi,txfo,pifo,xpp,drc,drlt,ap,xacc,itcb


In [ ]:
#| export
def clean(df: pd.DataFrame=None, # If None, downloads `download_vars`
          download_vars: List[str]=None, # If None, downloads `default_raw_vars`
          clean_kwargs: dict={}, # Params to pass to `pdm.setup_panel` other than `panel_ids`, `time_var`, and `freq`
          ) -> pd.DataFrame:
    
    if df is None: df = download(vars=download_vars)
    df = pdm.setup_panel(df, panel_ids='permno', time_var='datadate', freq='Y', **clean_kwargs)
    return df 

In [ ]:
#| eval: false
df = clean(raw)

In [ ]:
#| eval: false
df.head(0) 

,,datadate,dtdate,permco,iid,gvkey,cusip,cik,tic,fyear,fyr,...,intan,pi,txfo,pifo,xpp,drc,drlt,ap,xacc,itcb
permno,Ydate,,,,,,,,,,,,,,,,,,,,,


In [ ]:
#| export
def book_equity(df: pd.DataFrame=None, # If None, downloads (and cleans) only required vars
                add_itcb=False,
                list_reqs: bool=False # If true, just returns a list of the required variables
                ) -> pd.DataFrame:

    reqs = ['at', 'lt', 'seq', 'ceq', 'txditc', 'pstk', 'pstkrv', 'pstkl', 'itcb']
    if list_reqs: return reqs
    if df is None: df = clean(download_vars=reqs)
    df = df[reqs].copy()

    df['pstk'] = df['pstk'].fillna(0)
    df['pref_stock'] = np.where(df['pstkrv'].isnull(), df['pstkl'], df['pstkrv'])
    df['pref_stock'] = np.where(df['pref_stock'].isnull(),df['pstk'], df['pref_stock'])

    df['shreq'] = np.where(df['seq'].isnull(), df['ceq'] + df['pstk'], df['seq'])
    df['shreq'] = np.where(df['shreq'].isnull(), df['at'] - df['lt'], df['shreq'])

    df['bookeq'] = df['shreq'] + df['txditc'].fillna(0) - df['pref_stock']
    if add_itcb: df['bookeq'] = df['bookeq'] + df['itcb'].fillna(0)
    
    return df[['bookeq','shreq','pref_stock']].copy()

In [ ]:
#| eval: false
book_equity(list_reqs=True)

['at', 'lt', 'seq', 'ceq', 'txditc', 'pstk', 'pstkrv', 'pstkl', 'itcb']

In [ ]:
#| eval: false
beq = book_equity(df)

In [ ]:
#| eval: false
beq.head(0)

,,bookeq,shreq,pref_stock
permno,Ydate,,,


In [ ]:
#| eval: false
beq_from_scratch = book_equity()

Loading library list...
Done


In [ ]:
#| eval: false
beq_from_scratch.head(0)

,,bookeq,shreq,pref_stock
permno,Ydate,,,


In [ ]:
#| export 
def investment_vars(df: pd.DataFrame=None, # If None, downloads (and cleans) only required vars 
                    list_reqs: bool=False # If true, just returns a list of the required variables
                    ) -> pd.DataFrame:
    
    reqs = ['ppent','capx','at']
    if list_reqs: return reqs
    df = df[reqs].copy()

    df['ppentpch'] = pdm.rpct_change(df['ppent'])
    df['capx2la'] = df['capx'] / pdm.lag(df['at'])

    return df[['ppentpch','capx2la']].copy()

In [ ]:
#| eval: false
inv = investment_vars(df)

In [ ]:
#| eval: false
inv.head(0)

,,ppentpch,capx2la
permno,Ydate,,


In [ ]:
#| export 
def profitability_vars(df: pd.DataFrame, 
                    list_reqs: bool=False # If true, just returns a list of the required variables
                    ) -> pd.DataFrame:
    
    reqs = ['ib','at']
    if list_reqs: return reqs
    df = df[reqs].copy()

    df['roa'] = df['ib'] / df['at']

    return df[['roa']].copy()

In [ ]:
#| eval: false
prof = profitability_vars(df)

In [ ]:
#| eval: false
prof.head(0)

,,roa
permno,Ydate,


In [ ]:
#| export 
def cashflow_vars(df: pd.DataFrame, 
                    list_reqs: bool=False # If true, just returns a list of the required variables
                    ) -> pd.DataFrame:
    
    reqs = ['dtdate','oancf','ib','dp','at']
    if list_reqs: return reqs
    df = df[reqs].copy()

    df['cflow2la_is'] = (df['ib']+df['dp']) / pdm.lag(df['at'])
    df['cflow2la_cfs'] = df['oancf'] / pdm.lag(df['at'])
    df['cflow2la_full'] = np.where(df.dtdate.dt.year<1987, df['cflow2la_is'], df['cflow2la_cfs'])
    
    return df[['cflow2la_is', 'cflow2la_cfs', 'cflow2la_full']].copy()

In [ ]:
#| eval: false
cflow = cashflow_vars(df)

In [ ]:
#| eval: false
cflow.head(0)

,,cflow2la_is,cflow2la_cfs,cflow2la_full
permno,Ydate,,,


In [ ]:
#| export 
def liquidity_vars(df: pd.DataFrame, 
                    list_reqs: bool=False # If true, just returns a list of the required variables
                    ) -> pd.DataFrame:
    
    reqs = ['che','at']
    if list_reqs: return reqs
    df = df[reqs].copy()

    df['cash2a'] = df['che'] / df['at']

    return df[['cash2a']].copy()

In [ ]:
#| eval: false
liq = liquidity_vars(df)

In [ ]:
#| eval: false
liq.head(0)

,,cash2a
permno,Ydate,


In [ ]:
#| export 
def leverage_vars(df: pd.DataFrame, 
                    list_reqs: bool=False # If true, just returns a list of the required variables
                    ) -> pd.DataFrame:
    
    reqs = ['dltt','dlc','at']
    if list_reqs: return reqs
    df = df[reqs].copy()

    df['booklev'] = (df['dltt'] + df['dlc']) / df['at']
    df.loc[df.booklev<0, 'booklev'] = 0
    df.loc[df.booklev>1, 'booklev'] = 1
        
    return df[['booklev']].copy()

In [ ]:
#| eval: false
lev = leverage_vars(df)

In [ ]:
#| eval: false
lev.head(0)

,,booklev
permno,Ydate,


In [ ]:
#| export 
def payout_vars(df: pd.DataFrame, 
                    list_reqs: bool=False # If true, just returns a list of the required variables
                    ) -> pd.DataFrame:
    
    reqs = ['dvc','prstkc','at']
    if list_reqs: return reqs
    df = df[reqs].copy()

    df['div2la'] = df['dvc'].fillna(0) / pdm.lag(df['at'])
    df['rep2la'] = df['prstkc'].fillna(0) / pdm.lag(df['at'])

    return df[['div2la','rep2la']].copy()

In [ ]:
#| eval: false
payout = payout_vars(df)

In [ ]:
#| eval: false
payout.head(0)

,,div2la,rep2la
permno,Ydate,,


In [ ]:
#| export 
def value_vars(df: pd.DataFrame, 
                list_reqs: bool=False # If true, just returns a list of the required variables
                ) -> pd.DataFrame:
    
    reqs = ['at','prcc_f','csho'] + [x for x in book_equity(list_reqs=True) if x not in ['at','prcc_f','csho']]
    if list_reqs: return reqs
    df = df[reqs].copy()

    beq = book_equity(df)[['bookeq']].copy()
    df = df.join(beq)

    df['tobinq'] = (df['at'] - df['bookeq'] + df['prcc_f'] * df['csho']) / df['at']

    return  df[['tobinq']].copy()

In [ ]:
#| eval: false
tobinq = value_vars(df)

In [ ]:
#| eval: false
tobinq.head(0)

,,tobinq
permno,Ydate,


In [ ]:
#| export
def issuance_vars(df: pd.DataFrame, 
                list_reqs: bool=False # If true, just returns a list of the required variables
                ) -> pd.DataFrame:
    
    reqs_subset = ['at','sstk','prstkc','dltis','dltr', 're', 'dlc','dltt']
    reqs = reqs_subset + [x for x in book_equity(list_reqs=True) if x not in reqs_subset]
    if list_reqs: return reqs
    df = df[reqs].copy()

    beq = book_equity(df)[['bookeq']].copy()
    df = df.join(beq)
    
    df['lag_at'] = pdm.lag(df['at'])

    df['equityiss_cfs'] = (df['sstk'].fillna(0) - df['prstkc'].fillna(0)) / df['lag_at']
    df['debtiss_cfs'] = (df['dltis'].fillna(0) - df['dltr'].fillna(0)) / df['lag_at']

    df['debtiss_bs'] = (pdm.rdiff(df['dltt']) + pdm.rdiff(df['dlc'].fillna(0))) / df['lag_at']

    df['equityiss_tot'] = (pdm.rdiff(df['bookeq']) - pdm.rdiff(df['re'])) / df['lag_at']
    df['debtiss_tot'] = (pdm.rdiff(df['at']) - pdm.rdiff(df['bookeq'])) / df['lag_at']

    return df[['equityiss_tot','equityiss_cfs', 'debtiss_tot', 'debtiss_cfs', 'debtiss_bs']].copy()

In [ ]:
#| eval: false
iss = issuance_vars(df)

In [ ]:
#| eval: false
iss.head(0)

,,equityiss_tot,equityiss_cfs,debtiss_tot,debtiss_cfs,debtiss_bs
permno,Ydate,,,,,


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()